# Modeling with Support Vector Regressor
- The support vector machine (SVM) is a data-classification algorithm that assigns new data elements to one of the labeled categories, it assumes that the data in question contains some possible target values. In Machine Learning or Data Science are quite familiar with the term Support Vector Machine, but Support Vector Regression (SVR) is a bit different from SVM. As the name suggests the SVR is a regression algorithm, so we can use SVR for working with continuous Values instead of Classification which is SVM. The basic idea of SVR is to find a function f(y) that has most ε deviation from the actually obtained target for the training data yi, and at the same time is as flat as possible. In other words, **we do not care about the errors as long as they are less than ε**. **This property determines the SVR to be less sensitive to outliers than the quadratic loss function.**

In [1]:
%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import warnings
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler

In [2]:
df_train = pd.read_csv('../input/train_macro_without_outliers.csv', index_col=0)
df_train_augmented = pd.read_csv('../input/train_macro_with_outliers.csv', index_col=0)
df_test = pd.read_csv('../input/test_macro.csv', index_col=0)

In [3]:
df_train.tail(2)

,usdrub,full_sq,life_sq,floor,num_room,kitch_sq,state,product_type,area_m,preschool_education_centers_raion,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,culture_objects_top_25,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,0_17_all,raion_build_count_with_material_info,build_count_brick,build_count_monolith,raion_build_count_with_builddate_info,build_count_before_1920,metro_min_avto,kindergarten_km,school_km,park_km,railroad_station_walk_min,railroad_station_avto_min,public_transport_station_min_walk,water_1line,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,big_road1_1line,big_road2_km,railroad_1line,zd_vokzaly_avto_km,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,office_count_500,office_sqm_500,cafe_count_500,cafe_count_500_price_1000,cafe_count_500_price_1500,leisure_count_500,office_count_1000,office_sqm_1000,cafe_count_1000,cafe_count_1000_na_price,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_high,leisure_count_1000,sport_count_1000,office_count_1500,office_sqm_1500,trc_count_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_high,mosque_count_1500,leisure_count_1500,sport_count_1500,green_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_max_price_avg,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_high,mosque_count_2000,sport_count_2000,market_count_2000,green_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,big_church_count_3000,church_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,room_size,avg_price_sub_area,price_doc
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30472,55.2655,64,32.1,5.1,2.1,11.1,2.0,Investment,6050064.566,6.1,8.1,0.1,5469.953703,2.1,1.1,11.1,no,4.1,5.1,no,no,no,yes,no,no,no,no,10896,185.0,4.1,9.1,186.0,0.977191,3.477814,0.203020,0.230667,1.772506,82.750340,9.128624,3.138330,no,8.940313,11.752036,12.872535,13.622569,0.960608,no,2.174001,no,15.303338,5.458660,19.591574,8.011139,0.718679,1.971656,6.417997,3.881523,2.711199,0.512813,0.731280,2.730674,2.374106,2.210500,1.625064,24.788893,2.428096,1.982450,2.440429,1.208672,1.304798,1.440017,0.230667,1.644053,0.576021,2.748055,2.088193,4.119706,1.800186,0.134566,satisfactory,0.1,0.1,3.1,0.1,3.1,0.1,0.1,0.1,13.1,0.1,6.1,5.1,0.1,0.1,10.1,1.1,37800.1,1.1,42.1,646.34,1097.56,871.95,1.1,15.1,13.1,8.1,5.1,0.1,0.1,0.1,15.1,32.00,2.1,107800.1,10.1,136296.1,67.1,1195.31,3.1,17.1,23.1,15.1,9.1,0.1,0.1,18.1,2.1,30.31,15.1,473168.1,25.1,481350.1,2.1,17.1,2.1,33.1,4.1,30.36,39.1,1225712.1,45.1,1464521.1,6.1,31.1,1.1,4.1,65.1,7.1,16.1,1.392420e+07,13500000
30473,55.2655,43,28.1,1.1,2.1,6.1,2.0,Investment,4395332.782,4.1,4.1,0.1,3184.953703,2.1,0.1,7.1,no,5.1,1.1,no,no,no,yes,n

In [4]:
df_train_augmented.tail(2)

,usdrub,full_sq,life_sq,floor,num_room,kitch_sq,state,product_type,area_m,preschool_education_centers_raion,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,culture_objects_top_25,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,0_17_all,raion_build_count_with_material_info,build_count_brick,build_count_monolith,raion_build_count_with_builddate_info,build_count_before_1920,metro_min_avto,kindergarten_km,school_km,park_km,railroad_station_walk_min,railroad_station_avto_min,public_transport_station_min_walk,water_1line,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,big_road1_1line,big_road2_km,railroad_1line,zd_vokzaly_avto_km,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,office_count_500,office_sqm_500,cafe_count_500,cafe_count_500_price_1000,cafe_count_500_price_1500,leisure_count_500,office_count_1000,office_sqm_1000,cafe_count_1000,cafe_count_1000_na_price,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_high,leisure_count_1000,sport_count_1000,office_count_1500,office_sqm_1500,trc_count_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_high,mosque_count_1500,leisure_count_1500,sport_count_1500,green_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_max_price_avg,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_high,mosque_count_2000,sport_count_2000,market_count_2000,green_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,big_church_count_3000,church_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,room_size,avg_price_sub_area,price_doc
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30472,55.2655,64,32.1,5.1,2.1,11.1,2.0,Investment,6050064.566,6.1,8.1,0.1,5469.953703,2.1,1.1,11.1,no,4.1,5.1,no,no,no,yes,no,no,no,no,10896,185.0,4.1,9.1,186.0,0.977191,3.477814,0.203020,0.230667,1.772506,82.750340,9.128624,3.138330,no,8.940313,11.752036,12.872535,13.622569,0.960608,no,2.174001,no,15.303338,5.458660,19.591574,8.011139,0.718679,1.971656,6.417997,3.881523,2.711199,0.512813,0.731280,2.730674,2.374106,2.210500,1.625064,24.788893,2.428096,1.982450,2.440429,1.208672,1.304798,1.440017,0.230667,1.644053,0.576021,2.748055,2.088193,4.119706,1.800186,0.134566,satisfactory,0.1,0.1,3.1,0.1,3.1,0.1,0.1,0.1,13.1,0.1,6.1,5.1,0.1,0.1,10.1,1.1,37800.1,1.1,42.1,646.34,1097.56,871.95,1.1,15.1,13.1,8.1,5.1,0.1,0.1,0.1,15.1,32.00,2.1,107800.1,10.1,136296.1,67.1,1195.31,3.1,17.1,23.1,15.1,9.1,0.1,0.1,18.1,2.1,30.31,15.1,473168.1,25.1,481350.1,2.1,17.1,2.1,33.1,4.1,30.36,39.1,1225712.1,45.1,1464521.1,6.1,31.1,1.1,4.1,65.1,7.1,16.1,1.392420e+07,13500000
30473,55.2655,43,28.1,1.1,2.1,6.1,2.0,Investment,4395332.782,4.1,4.1,0.1,3184.953703,2.1,0.1,7.1,no,5.1,1.1,no,no,no,yes,n

In [5]:
df_test.tail(2)

,usdrub,full_sq,life_sq,floor,num_room,kitch_sq,state,product_type,area_m,raion_popul,preschool_education_centers_raion,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,culture_objects_top_25,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,young_all,work_all,ekder_all,0_17_all,raion_build_count_with_material_info,build_count_brick,build_count_monolith,raion_build_count_with_builddate_info,build_count_before_1920,metro_min_avto,kindergarten_km,school_km,park_km,railroad_station_walk_min,railroad_station_avto_min,public_transport_station_min_walk,water_1line,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,big_road1_1line,big_road2_km,railroad_1line,zd_vokzaly_avto_km,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,office_count_500,office_sqm_500,cafe_count_500,cafe_count_500_price_1000,cafe_count_500_price_1500,leisure_count_500,office_count_1000,office_sqm_1000,cafe_count_1000,cafe_count_1000_na_price,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_high,leisure_count_1000,sport_count_1000,office_count_1500,office_sqm_1500,trc_count_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_high,mosque_count_1500,leisure_count_1500,sport_count_1500,green_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_max_price_avg,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_high,mosque_count_2000,sport_count_2000,market_count_2000,green_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,cafe_count_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,room_size,avg_price_sub_area
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38134,65.6745,34.8,19.8,8.0,1.0,6.4,2.0,Investment,7128794.338,145576,7,7,1,1031.546494,1,0,7,no,6,0,no,no,no,no,no,yes,no,no,13595,104635,27346,14976,195.0,0.0,3.0,195.0,0.0,1.469263,0.073023,0.208540,2.950264,60.226932,6.164570,1.392524,no,11.538742,14.88307,16.528376,17.137752,1.425847,no,1.425847,no,17.433375,2.160649,6.997322,5.259407,3.258864,2.011375,6.365003,4.649813,1.280650,0.542683,1.277750,1.116351,6.449755,3.994409,2.596009,4.655112,1.350505,3.679888,1.795903,0.469357,1.467622,0.676312,0.208540,2.411682,0.331122,8.247379,12.564484,3.127103,3.618234,0.322872,poor,0,0,3,2,0,0,0,0,20,1,9,4,0,0,6,2,54500,6,27,630

## 1. Preprocessing
- Label Encoding
- Log nomalization
- Standard scaling

## 2. Support Vector Regreesor

In [6]:
cate_features = list(set(df_train.columns) - set(df_train._get_numeric_data().columns))
numeric_features = list(df_train.columns.drop(cate_features + ['price_doc']).values)

## 1. Preprocessing
- Label Encoding
- Log nomalization
- Standard scaling

### Label Encoding

In [7]:
df_train_cat = df_train[cate_features]
df_train_cat1 = df_train_cat
encode = OneHotEncoder(sparse=False)

for col in cate_features:
    encode.fit(df_train_cat[[col]])
    transform = encode.transform(df_train_cat[[col]])
    
    transform = pd.DataFrame(transform, 
                             columns=[(col+"_"+str(i)) for i in df_train_cat[col].value_counts().index])
    transform = transform.set_index(df_train_cat.index.values)
    
    df_train_cat1 = pd.concat([df_train_cat1, transform], axis=1)
    df_train_cat1 = df_train_cat1.drop(col, 1)

In [ ]:
df_train_cat1.tail(2)

### Log nomalizaion

In [8]:
from scipy import stats

# Log Normalization of Numeric Features
for column in numeric_features + ['price_doc']:
    if stats.skew(df_train[column].values) > 1:
        df_train[column] = np.log(df_train[column] + 1)  
        df_train_augmented[column] = np.log(df_train_augmented[column] + 1)
        if column in df_test.columns.values:
            df_test[column]  = np.log(df_test[column] + 1)

### Standard scaling

In [9]:
# 평균 0 표준편차 1이 되도록 스케일링
train_scaler = StandardScaler()
train_scaler.fit(df_train[numeric_features])

scaled_numeric_train_X = train_scaler.transform(df_train[numeric_features])
df_scaled_numeric_train_X = pd.DataFrame(scaled_numeric_train_X, index=df_train.index, columns=numeric_features)
df_train = pd.concat([df_scaled_numeric_train_X, df_train_cat1, df_train['price_doc']], axis=1)

# scaled_numeric_test_X = train_scaler.transform(df_test[numeric_features])
# df_scaled_numeric_test_X = pd.DataFrame(scaled_numeric_test_X, index=df_test.index, columns=numeric_features)
# df_test = pd.concat([df_scaled_numeric_test_X, df_train[cate_features]], axis=1)

# scaled_numeric_train_X = train_scaler.transform(df_train_augmented[numeric_features])
# df_scaled_numeric_train_X = pd.DataFrame(scaled_numeric_train_X, index=df_train_augmented.index, columns=numeric_features)
# df_train = pd.concat([df_scaled_numeric_train_X, df_train_augmented[cate_features]],axis=1)

/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [10]:
df_train.tail(2)

,usdrub,full_sq,life_sq,floor,num_room,kitch_sq,state,area_m,preschool_education_centers_raion,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,shopping_centers_raion,office_raion,0_17_all,raion_build_count_with_material_info,build_count_brick,build_count_monolith,raion_build_count_with_builddate_info,build_count_before_1920,metro_min_avto,kindergarten_km,school_km,park_km,railroad_station_walk_min,railroad_station_avto_min,public_transport_station_min_walk,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,big_road2_km,zd_vokzaly_avto_km,bus_terminal_avto_km,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,office_count_500,office_sqm_500,cafe_count_500,cafe_count_500_price_1000,cafe_count_500_price_1500,leisure_count_500,office_count_1000,office_sqm_1000,cafe_count_1000,cafe_count_1000_na_price,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_high,leisure_count_1000,sport_count_1000,office_count_1500,office_sqm_1500,trc_count_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_high,mosque_count_1500,leisure_count_1500,sport_count_1500,green_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_max_price_avg,cafe_count_2000_na_price,cafe_count_2000_price_500,cafe_count_2000_price_1000,cafe_count_2000_price_1500,cafe_count_2000_price_2500,cafe_count_2000_price_high,mosque_count_2000,sport_count_2000,market_count_2000,green_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,big_church_count_3000,church_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,room_size,avg_price_sub_area,railroad_terminal_raion_no,railroad_terminal_raion_yes,nuclear_reactor_raion_no,nuclear_reactor_raion_yes,detention_facility_raion_no,detention_facility_raion_yes,ecology_no data,ecology_good,ecology_poor,ecology_excellent,ecology_satisfactory,culture_objects_top_25_no,culture_objects_top_25_yes,thermal_power_plant_raion_no,thermal_power_plant_raion_yes,water_1line_no,water_1line_yes,oil_chemistry_raion_no,oil_chemistry_raion_yes,big_market_raion_no,big_market_raion_yes,incineration_raion_no,incineration_raion_yes,big_road1_1line_no,big_road1_1line_yes,radiation_raion_no,radiation_raion_yes,product_type_Investment,product_type_OwnerOccupier,railroad_1line_no,railroad_1line_yes,price_doc
30472,1.834105,0.752733,0.089728,-0.521740,0.179317,1.130801,-0.03416,-0.901343,0.658269,0.975016,-0.322377,1.762127,0.779084,2.514275,0.944188,0.418008,0.593391,-0.141997,0.035858,-1.258351,0.518236,0.042921,-0.507295,-0.031417,-0.677593,-0.831873,-0.244537,0.919986,0.920955,0.180626,-0.065891,-0.087813,-0.055495,-0.087672,-0.740215,-0.689143,0.006953,-0.588652,0.173815,-0.199380,-1.175996,-0.200835,0.034556,-0.031246,-0.222101,-0.677684,-1.576650,-0.880035,-1.372364,-0.445854,-0.365496,0.943320,-0.134618,-0.997820,-0.177768,-0.046449,-0.259476,0.237768,-0.856130,-0.115491,-0.840906,-1.138720,-1.630298,-0.407877,-0.987536,-0.942310,-0.430364,-0.48802,0.633537,-0.617938,1.910940,-0.205669,-0.680333,-0.882595,0.839032,-0.530255,1.108213,1.157861,-0.1904,-0.341651,1.759636,-0.254122,0.692382,-0.511

## 2. Support Vector Regreesor

In [11]:
from sklearn.svm import SVR
import numpy as np


y= df_train.iloc[:, -1]
X = df_train.iloc[:, :-1]
clf = SVR(gamma='scale', C=1.0, epsilon=0.2)
clf.fit(X, y) 
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='scale',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [12]:
score

NameError: name 'score' is not defined